### Importar en los notebooks
```python
import sys
sys.path.append('..')
from utils.paths import PATHS, PARAMS, get_file_path, get_data_path


### Acceso directo
```python
df = pd.read_csv(get_file_path('cleaned', 'exportaciones'))
paises = pd.read_csv(get_file_path('mappings', 'paises'))

### O acceso por carpeta
```python
raw_folder = get_data_path('raw')
cleaned_folder = get_data_path('cleaned')

### Construir ruta personalizada
```python
mi_archivo = get_data_path('raw', 'nuevo_archivo.csv')

### Usando `get_file_path`

In [1]:
import pandas as pd
import sys
sys.path.append('..')
from utils.paths import get_file_path


In [11]:
df = pd.read_csv(get_file_path('mappings', 'paises'))

In [12]:
df.head()

,Código,Nombre
0,13,Afganistán
1,15,Islas Åland
2,17,Albania
3,23,Alemania
4,24,Antártida


In [3]:
df = pd.read_csv(get_file_path('cleaned', 'exportaciones'), low_memory=False)

In [ ]:
dist_temp = df.groupby('AÑO')['FOBDOL'].sum().reset_index()

In [2]:
import polars as pl
import sys
sys.path.append('..')
from utils.paths import get_file_path

df = pl.scan_csv(
    get_file_path('cleaned', 'exportaciones'),
    low_memory=True
).collect()

In [19]:
temporal = df.group_by("AÑO").agg(pl.col("FOBDOL").sum()).sort("AÑO")

In [4]:
print(temporal)

shape: (21, 2)
┌──────┬───────────┐
│ AÑO  ┆ FOBDOL    │
│ ---  ┆ ---       │
│ i64  ┆ f64       │
╞══════╪═══════════╡
│ 2004 ┆ 1.6788e10 │
│ 2005 ┆ 2.1146e10 │
│ 2006 ┆ 2.4512e10 │
│ 2007 ┆ 3.0279e10 │
│ 2008 ┆ 3.6786e10 │
│ …    ┆ …         │
│ 2020 ┆ 3.1056e10 │
│ 2021 ┆ 4.1390e10 │
│ 2022 ┆ 5.6910e10 │
│ 2023 ┆ 4.9769e10 │
│ 2024 ┆ 4.9557e10 │
└──────┴───────────┘


In [5]:
print(type(df))

<class 'polars.dataframe.frame.DataFrame'>


In [6]:
import plotly.express as px

In [77]:
def aplicar_estilo(fig):
    fig.update_layout(
        title_font=dict(size=24, family="Arial", color="black"),
        xaxis_tickangle=-45,
        xaxis_tickfont=dict(size=12),
        yaxis_tickfont=dict(size=12),
        plot_bgcolor="white",
        paper_bgcolor="white",
        showlegend=False,
        margin=dict(t=80, l=40, r=20, b=120),
    )
    return fig


In [80]:
fig = px.line(temporal,
              x='AÑO',
              y='FOBDOL',
              title='Gráfico de líneas del valor total exportado por año',
              labels={"AÑO": "Año", "FOBDOL": "FOB Dólares"}
              )
fig = aplicar_estilo(fig)
fig.show()

In [13]:
top_paises = df.group_by("PAIS").agg(pl.col("FOBDOL").sum()).sort("FOBDOL", descending=True).head(10)

In [14]:
print(top_paises)

shape: (10, 2)
┌────────────────┬───────────┐
│ PAIS           ┆ FOBDOL    │
│ ---            ┆ ---       │
│ str            ┆ f64       │
╞════════════════╪═══════════╡
│ Estados Unidos ┆ 2.7012e11 │
│ China          ┆ 4.8965e10 │
│ Panamá         ┆ 4.5006e10 │
│ Venezuela      ┆ 3.7167e10 │
│ Ecuador        ┆ 3.4099e10 │
│ Países Bajos   ┆ 2.9687e10 │
│ Brasil         ┆ 2.4973e10 │
│ España         ┆ 2.4097e10 │
│ Perú           ┆ 2.1906e10 │
│ México         ┆ 2.1640e10 │
└────────────────┴───────────┘


In [83]:
fig = px.bar(top_paises,
             x='PAIS',
             y='FOBDOL',
             title='Top 10 países destino',
             labels={"PAIS": "País", "FOBDOL": "FOB Dólares"}
             )
fig = aplicar_estilo(fig)
fig.show()

In [51]:
top_productos = df.group_by("POSARA").agg(pl.col("FOBDOL").sum()).sort("FOBDOL", descending=True).head(10)

In [46]:
print(top_productos)

shape: (10, 2)
┌────────────┬───────────┐
│ POSARA     ┆ FOBDOL    │
│ ---        ┆ ---       │
│ i64        ┆ f64       │
╞════════════╪═══════════╡
│ 2709000000 ┆ 2.6994e11 │
│ 2701120010 ┆ 1.0871e11 │
│ 901119000  ┆ 4.6323e10 │
│ 7108120000 ┆ 3.7659e10 │
│ 2710192200 ┆ 2.4022e10 │
│ 7202600000 ┆ 1.4968e10 │
│ 2704001000 ┆ 1.2091e10 │
│ 2710192100 ┆ 1.1212e10 │
│ 803901100  ┆ 1.1023e10 │
│ 603199090  ┆ 6.7105e9  │
└────────────┴───────────┘


In [53]:
top_productos = top_productos.to_pandas()

In [48]:
print(type(top_productos))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
productos = {
    2709000000: 'Petróleos crudos e hidrocarburos',
    2701120010: 'Carbón bituminoso y briquetas',
    901119000: 'Café sin tostar',
    7108120000: 'Gemas y metales preciosos',
    2710192200: 'Fuelóleos y gasóleos pesados',
    7202600000: 'Ferroaleaciones de níquel',
    2704001000: 'Coques y semicoques de hulla',
    2710192100: 'Aceites de petróleo (≥70 %)',
    803901100: 'Bananas y plátanos frescos',
    603199090: 'Flores y capullos cortados, frescos'
}

top_productos = top_productos.rename(columns={'POSARA': 'PRODUCTOS'})
top_productos['PRODUCTOS'] = top_productos['PRODUCTOS'].map(productos)

In [84]:
fig = px.bar(top_productos,
             x='PRODUCTOS',
             y='FOBDOL',
             title='Top 10 productos',
             labels={"PRODUCTOS": "Productos", "FOBDOL": "FOB Dólares"}
             )
fig = aplicar_estilo(fig)
fig.show()